In [ ]:
!pip install pypdf

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
## Embedding
!pip install install sentence_transformers

In [ ]:
!pip install llama_index

In [ ]:
pip install llama-index-llms-huggingface

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [ ]:
documents=SimpleDirectoryReader("./data").load_data()
documents

In [ ]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")


In [ ]:
!huggingface-cli login

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)


In [ ]:
pip show langchain

In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# from llama_index import ServiceContext
from llama_index.legacy.embeddings.langchain import LangchainEmbedding
from llama_index.core import Settings

In [ ]:
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

In [ ]:
from llama_index.core.node_parser import SentenceSplitter


In [ ]:
Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=1024)


In [ ]:
# Create a vector store index with the settings
index = VectorStoreIndex.from_documents(
    documents, embed_model=Settings.embed_model
)

In [ ]:
index

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
response=query_engine.query("what is attention is all you need?")

In [ ]:
print(response)